In [64]:
!pip install geopy

     -------------------------------------- 119.8/119.8 kB 7.3 MB/s eta 0:00:00
     ---------------------------------------- 40.3/40.3 kB 2.0 MB/s eta 0:00:00



[notice] A new release of pip available: 22.3 -> 22.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [65]:
from bs4 import BeautifulSoup
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from geopy.geocoders import Nominatim
import pandas as pd
import time
import re
import requests

### 1. 부산시 대학교 주소 크롤링

- 부산시 소재 대학교 파일 불러와서 네이버 지도에서 검색후 다시 주소열 생성

In [56]:
uni = pd.read_excel('대학교.xlsx',index_col=0).reset_index()
uni.head()

,학교이름
0,경성대학교
1,고신대학교 영도캠퍼스
2,고신대학교 송도캠퍼스
3,동명대학교
4,동서대학교


In [57]:
uni['주소지'] = None

In [60]:
uni.주소지.head()

0    None
1    None
2    None
3    None
4    None
Name: 주소지, dtype: object

- 네이버지도에서 크롤링하기
- 간단한 주소만

In [4]:
map = list()

for i in range(len(uni.학교이름)):
    # USB: usb_device_handle_win.cc:1048 Failed to read descriptor from node connection: 시스템에 부착된 장치가 작동하지 않습니다. (0x1F)
    options = webdriver.ChromeOptions()
    options.add_experimental_option('excludeSwitches', ['enable-logging'])
    wd = webdriver.Chrome('chromedriver', options=options)
    
    row = uni.학교이름.iloc[i]
    url = (f'https://map.naver.com/v5/search/{row}?c=14368636.8623293,4195641.9152365,15,0,0,0,dh')
    wd.get(url)

    # 네이버 지도 신버전 html 불러오기 실패 -> iframe에 접근하는 코드
    wd.switch_to.frame('searchIframe')

    html = wd.page_source
    time.sleep(1)
    soup=BeautifulSoup(html, 'html.parser')

    map.append(soup.select('.Q8Zql')[0].get_text())

    print(map[i])

wd.close()

부산 남구 대연동
부산 영도구 동삼동
부산 서구 암남동
부산 남구 용당동
부산 사상구 주례동
부산 부산진구 양정동
부산 해운대구 우동
부산 사하구 하단동
부산 서구 동대신동3가
부산 서구 부민동3가
부산 부산진구 가야동
부산 부산진구 양정동
부산 남구 대연동
부산 남구 용당동
부산 금정구 부곡동
부산 연제구 거제동
부산 금정구 장전동
부산 서구 아미동1가
부산 금정구 남산동
부산 사상구 괘법동
부산 영도구 동삼동
부산 사상구 주례동
부산 금정구 부곡동
부산 부산진구 양정동
부산 부산진구 전포동
부산 사하구 괴정동
부산 연제구 연산동
부산 북구 구포동
부산 부산진구 양정동
부산 남구 대연동


- 클릭으로 상세주소 뽑아오기
- o8CtQ

In [61]:
map = list()

for i in range(len(uni.학교이름)):
    # USB: usb_device_handle_win.cc:1048 Failed to read descriptor from node connection: 시스템에 부착된 장치가 작동하지 않습니다. (0x1F)
    options = webdriver.ChromeOptions()
    options.add_experimental_option('excludeSwitches', ['enable-logging'])
    wd = webdriver.Chrome('chromedriver', options=options)
    
    row = uni.학교이름.iloc[i]
    url = (f'https://map.naver.com/v5/search/{row}?c=14368636.8623293,4195641.9152365,15,0,0,0,dh')
    wd.get(url)

    # 네이버 지도 신버전 html 불러오기 실패 -> iframe에 접근하는 코드
    wd.switch_to.default_content()
    wd.switch_to.frame('searchIframe')

    html = wd.page_source
    soup=BeautifulSoup(html, 'html.parser')
    time.sleep(1)

    # 
    click = wd.find_element(By.CLASS_NAME, 'FEhi4')
    try:
        click.click()
    except:
        continue
    
    html = wd.page_source
    soup=BeautifulSoup(html, 'html.parser')
    time.sleep(1)

    uni['주소지'].iloc[i] = soup.select('.o8CtQ')[0].text[3:-2]

    print(uni['주소지'].iloc[i])
    
wd.close()

부산 남구 수영로 309 경성대학교
부산 영도구 와치로 194
부산 서구 성산길 46
부산 남구 신선로 428
부산 사상구 주례로 47
부산 해운대구 센텀중앙로 55
부산 사하구 낙동남로1373번길 7
부산 서구 망양로111번길 65 경남고등학교
부산 서구 고운들로35번길 12
부산 부산진구 엄광로 176
부산 부산진구 양정로 52-57 동의대학교(한의학관) C동
부산 남구 수영로 298
부산 남구 신선로 365 부경대학교
부산 금정구 오륜대로 57
부산 연제구 교대로 24 부산교육대학교
부산 금정구 부산대학로63번길 2
부산 서구 구덕로 179
부산 금정구 금샘로485번길 65 부산외국어대학교
부산 사상구 백양대로700번길 140 신라대학교
부산 영도구 태종로 727
부산 사상구 주례로 45
부산 금정구 동부곡로27번길 88
산 부산진구 전포동 12-1
부산 사하구 낙동대로 164 한라빌딩2층
부산 연제구 고분로 181
부산 북구 시랑로132번길 88 부산과학기술대학교
부산 남구 못골번영로71번길 74 부산예술대학


In [63]:
uni.head()

,학교이름,주소지
0,경성대학교,부산 남구 수영로 309 경성대학교
1,고신대학교 영도캠퍼스,부산 영도구 와치로 194
2,고신대학교 송도캠퍼스,부산 서구 성산길 46
3,동명대학교,부산 남구 신선로 428
4,동서대학교,부산 사상구 주례로 47


#### 위도 경도 찾기

In [112]:
uni.to_csv('./대학주소지.csv', encoding='cp949')

In [113]:
uni_add = pd.read_csv('./대학주소지.csv', encoding='cp949', index_col=0)

In [114]:
uni_add['x'] = None
uni_add['y'] = None

In [115]:
geo_local = Nominatim(user_agent='South Korea')

def geocoding(address):
    try:
        geo = geo_local.geocode(address)
        x_y = [geo.latitude, geo.longitude]
        return x_y
    except:
        return[0,0]

latitude = []
longitude = []

for i in uni_add['주소지']:
    latitude.append(geocoding(i)[0])
    longitude.append(geocoding(i)[1])


In [116]:
uni_add['x'] = latitude
uni_add['y'] = longitude

In [120]:
uni_add.to_csv('./대학주소지위경도.csv', encoding='cp949')
uni_addxy = pd.read_csv('./대학주소지위경도.csv', encoding='cp949', index_col=0)

In [121]:
uni_addxy

,학교이름,주소지,x,y
0,경성대학교,부산 남구 수영로 309 경성대학교,35.136086,129.097546
1,고신대학교 영도캠퍼스,부산 영도구 와치로 194,35.071839,129.073801
2,고신대학교 송도캠퍼스,부산 서구 성산길 46,35.080562,129.016959
3,동명대학교,부산 남구 신선로 428,35.122949,129.101286
4,동서대학교,부산 사상구 주례로 47,35.150292,129.011425
5,동서대학교 선교복지대학원,NaN,46.314475,11.048029
6,동서대학교 센텀캠퍼스,부산 해운대구 센텀중앙로 55,35.178629,129.122958
7,동아대학교 승학캠퍼스,부산 사하구 낙동남로1373번길 7,35.107309,128.961901
8,동아대학교 구덕캠퍼스,부산 서구 망양로111번길 65 경남고등학교,0.000000,0.000000
9,동아대학교 부민캠퍼스,부산 서구 고운들로35번길 12,35.104311,129.018273


In [122]:
for i in range(len(uni_addxy['x'])):
    if uni_addxy['x'].iloc[i] == 0 :
        uni_addxy['주소지'].iloc[i] = ' '.join(uni_addxy['주소지'].iloc[i].split()[:-1])
        print(uni_addxy['주소지'].iloc[i])
        print(i)

부산 서구 망양로111번길 65
8
부산 부산진구 양정로 52-57 동의대학교(한의학관)
11
부산 연제구 교대로 24
15
부산 사상구 백양대로700번길 140
19
산 부산진구 전포동
24
부산 사하구 낙동대로 164
25
부산 북구 시랑로132번길 88
27
부산 남구 못골번영로71번길 74
29


C:\Users\admin\AppData\Local\Temp\ipykernel_14052\3118124808.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uni_addxy['주소지'].iloc[i] = ' '.join(uni_addxy['주소지'].iloc[i].split()[:-1])


In [126]:
uni_addxy['주소지'].iloc[11] = '부산 부산진구 양정로 52-57'
uni_addxy['주소지'].iloc[24] = '부산 부산진구 전포동 12-1'
uni_addxy['주소지'].iloc[28] = '부산 부산진구 진남로 506'
uni_addxy

C:\Users\admin\AppData\Local\Temp\ipykernel_14052\3304079714.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uni_addxy['주소지'].iloc[11] = '부산 부산진구 양정로 52-57'
C:\Users\admin\AppData\Local\Temp\ipykernel_14052\3304079714.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uni_addxy['주소지'].iloc[24] = '부산 부산진구 전포동 12-1'
C:\Users\admin\AppData\Local\Temp\ipykernel_14052\3304079714.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uni_a

,학교이름,주소지,x,y
0,경성대학교,부산 남구 수영로 309 경성대학교,35.136086,129.097546
1,고신대학교 영도캠퍼스,부산 영도구 와치로 194,35.071839,129.073801
2,고신대학교 송도캠퍼스,부산 서구 성산길 46,35.080562,129.016959
3,동명대학교,부산 남구 신선로 428,35.122949,129.101286
4,동서대학교,부산 사상구 주례로 47,35.150292,129.011425
5,동서대학교 선교복지대학원,NaN,46.314475,11.048029
6,동서대학교 센텀캠퍼스,부산 해운대구 센텀중앙로 55,35.178629,129.122958
7,동아대학교 승학캠퍼스,부산 사하구 낙동남로1373번길 7,35.107309,128.961901
8,동아대학교 구덕캠퍼스,부산 서구 망양로111번길 65,0.000000,0.000000
9,동아대학교 부민캠퍼스,부산 서구 고운들로35번길 12,35.104311,129.018273


In [132]:
uni_addxy = uni_addxy.dropna()

In [133]:
geo_local = Nominatim(user_agent='South Korea')

def geocoding(address):
    try:
        geo = geo_local.geocode(address)
        x_y = [geo.latitude, geo.longitude]
        return x_y
    except:
        return[0,0]

latitude = []
longitude = []

for i in uni_addxy['주소지']:
    latitude.append(geocoding(i)[0])
    longitude.append(geocoding(i)[1])


In [134]:
uni_addxy['x'] = latitude
uni_addxy['y'] = longitude

C:\Users\admin\AppData\Local\Temp\ipykernel_14052\21876488.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uni_addxy['x'] = latitude
C:\Users\admin\AppData\Local\Temp\ipykernel_14052\21876488.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uni_addxy['y'] = longitude


In [139]:
uni_addxy = uni_addxy.rename(columns={'x':'위도', 'y':'경도'})
uni_addxy

,학교이름,주소지,위도,경도
0,경성대학교,부산 남구 수영로 309 경성대학교,35.136086,129.097546
1,고신대학교 영도캠퍼스,부산 영도구 와치로 194,35.071839,129.073801
2,고신대학교 송도캠퍼스,부산 서구 성산길 46,35.080562,129.016959
3,동명대학교,부산 남구 신선로 428,35.122949,129.101286
4,동서대학교,부산 사상구 주례로 47,35.150292,129.011425
6,동서대학교 센텀캠퍼스,부산 해운대구 센텀중앙로 55,35.178629,129.122958
7,동아대학교 승학캠퍼스,부산 사하구 낙동남로1373번길 7,35.107309,128.961901
8,동아대학교 구덕캠퍼스,부산 서구 망양로111번길 65,35.118573,129.019898
9,동아대학교 부민캠퍼스,부산 서구 고운들로35번길 12,35.104311,129.018273
10,동의대학교 가야캠퍼스,부산 부산진구 엄광로 176,35.143096,129.034164


In [140]:
uni_addxy.to_csv('./대학주소지위경도.csv', encoding='cp949')

### 2. 지하철역과 대학교 사이 거리 계산

In [138]:
from haversine import haversine

In [137]:
!pip install haversine


[notice] A new release of pip available: 22.3 -> 22.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


- 대학교와 역 사이의 거리 1.2 km

In [141]:
sub_area = pd.read_csv('./통합_사용_데이터_추가수정_handling.csv',encoding='cp949',index_col=0)[['역사명','역위도','역경도']]
sub_area['주변 대학교'] = 0
sub_area.head()

,역사명,역위도,역경도,주변 대학교
0,다대포해수욕장,35.048670,128.964100,0
1,다대포항,35.057820,128.971300,0
2,낫개,35.065265,128.979873,0
3,신장림,35.074433,128.977041,0
4,장림,35.081090,128.977500,0


In [142]:
for idx,sub_line in enumerate(sub_area.iloc):
    start = (sub_line[1],sub_line[2])
    for uni_line in uni_addxy.iloc:
        end = (uni_line[2],uni_line[3])
        length = haversine(start,end)
        if 1.2 > length:
            sub_area['주변 대학교'].iloc[idx] = 1

C:\Users\admin\AppData\Local\Temp\ipykernel_14052\101705302.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sub_area['주변 대학교'].iloc[idx] = 1


In [154]:
len(sub_area[sub_area['주변 대학교']>0])

37

In [146]:
org_data = pd.read_csv('./통합_사용_데이터_추가수정_handling.csv', encoding='cp949', index_col=0)
org_data.head()

,역사명,동별,역위도,역경도,일평균승하차,업장수,1인 가구수,2인 가구수,환승,총 상가 수,...,20~40대 인구 수,일평균승하차 레벨,업장수 레벨,1인 가구수 레벨,2인 가구수 레벨,20~40대 인구 수 레벨,고속버스 터미널,기차역,여객 터미널,주변 관광지
0,다대포해수욕장,다대1동,35.048670,128.964100,7761,345,6277,4362,NaN,2.0,...,8485,41,69,88,96,97,NaN,NaN,NaN,3
1,다대포항,다대1동,35.057820,128.971300,6221,345,6277,4362,NaN,1.0,...,8485,31,69,88,96,97,NaN,NaN,NaN,1
2,낫개,다대2동,35.065265,128.979873,8327,246,4633,3170,NaN,1.0,...,5674,44,56,71,81,60,NaN,NaN,NaN,0
3,신장림,장림2동,35.074433,128.977041,5360,302,4160,3230,NaN,1.0,...,6894,21,64,62,86,78,NaN,NaN,NaN,0
4,장림,장림2동,35.081090,128.977500,5490,302,4160,3230,NaN,1.0,...,6894,23,64,62,86,78,NaN,NaN,NaN,0


In [148]:
sub_area = sub_area[['주변 대학교']]
sub_area

,주변 대학교
0,0
1,0
2,0
3,0
4,0
...,...
96,0
97,0
98,0
99,0


In [155]:
total = pd.concat([org_data,sub_area], axis = 1)
len(total[total['주변 대학교']>0])

37

In [156]:
total.to_csv('./통합_사용_데이터_추가수정_handling.csv', encoding = 'cp949')

### x. 주소지에서 동만 뽑아내기

- 동으로 구분하여 뽑아낼때 '동'으로 시작하는 주소는 따로 처리

In [6]:
dong = list()
for i in range(len(uni_add.주소지)):
    row = uni_add.주소지.iloc[i].split()[-1]

    # '동'으로 시작하는 주소 처리
    if re.match('동', row):
        dong.append(row)
    # 그 외 전체 주소 처리
    else:
        dong.append(row.split('동')[0])

uni_add['동별'] = dong
uni_add

,학교이름,주소지,동별
0,경성대학교,부산 남구 대연동,대연
1,고신대학교 영도캠퍼스,부산 영도구 동삼동,동삼동
2,고신대학교 송도캠퍼스,부산 서구 암남동,암남
3,동명대학교,부산 남구 용당동,용당
4,동서대학교,부산 사상구 주례동,주례
5,동서대학교 선교복지대학원,부산 부산진구 양정동,양정
6,동서대학교 센텀캠퍼스,부산 해운대구 우동,우
7,동아대학교 승학캠퍼스,부산 사하구 하단동,하단
8,동아대학교 구덕캠퍼스,부산 서구 동대신동3가,동대신동3가
9,동아대학교 부민캠퍼스,부산 서구 부민동3가,부민


In [7]:
uni_add['동별'] = uni_add['동별'].replace(['동삼동','동대신동3가'],['동삼','동대신'])
uni_add

,학교이름,주소지,동별
0,경성대학교,부산 남구 대연동,대연
1,고신대학교 영도캠퍼스,부산 영도구 동삼동,동삼
2,고신대학교 송도캠퍼스,부산 서구 암남동,암남
3,동명대학교,부산 남구 용당동,용당
4,동서대학교,부산 사상구 주례동,주례
5,동서대학교 선교복지대학원,부산 부산진구 양정동,양정
6,동서대학교 센텀캠퍼스,부산 해운대구 우동,우
7,동아대학교 승학캠퍼스,부산 사하구 하단동,하단
8,동아대학교 구덕캠퍼스,부산 서구 동대신동3가,동대신
9,동아대학교 부민캠퍼스,부산 서구 부민동3가,부민


In [9]:
uni_add.to_csv('./univ_dong.csv', encoding='cp949')